<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-11-02 06:05:24--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-11-02 06:05:24 (68.8 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [3]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-11-02 06:05:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.207, 192.178.219.207, 74.125.134.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2025-11-02 06:05:43 (103 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [12]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 1 
  (real disaster)
Text : 
 http://t.co/J8TYT1XRRK Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/9d4nAzOI94
---

Target : 0 
  not real disaster
Text : 
 OMEGA MEN Writer Explores Terrorism Religion In an 'Epic KYLE RAYNER Story' http://t.co/Hr88CWxPGz #Newsarama
---

Target : 1 
  (real disaster)
Text : 
 Militants attack police post in Udhampur; 2 SPOs injured | LiveMint http://t.co/Rptouz2iJs | http://t.co/69mLhfefhr #AllTheNews
---

Target : 0 
  not real disaster
Text : 
 I am not in danger Skyler. I AM THE DANGER.
---

Target : 0 
  not real disaster
Text : 
 Fun with my girls! @ Joe Cools - Durban Main Page https://t.co/AbnZQWlig1
---



In [13]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [14]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [16]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [17]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [18]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 0-day bug in fully patched OS X comes under active exploit to bypass password ... - Ars Technica http://t.co/F7OgzrNPfv 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [19]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [20]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [21]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 Canceling Deal for 2 Warships #France Agrees to Repay #Russia via @nytimes http://t.co/f2gwxEPrAk



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[-0.01057674, -0.01388558,  0.01806736, ...,  0.01663825,
          0.01673924, -0.04629226],
        [-0.01057674, -0.01388558,  0.01806736, ...,  0.01663825,
          0.01673924, -0.04629226],
        [-0.01419489,  0.00196706, -0.01609423, ..., -0.03847673,
          0.01035788,  0.0199081 ],
        ...,
        [-0.01813293, -0.02233012, -0.03890401, ..., -0.02798435,
          0.02484895, -0.04323708],
        [-0.01813293, -0.02233012, -0.03890401, ..., -0.02798435,
          0.02484895, -0.04323708],
        [-0.01813293, -0.02233012, -0.03890401, ..., -0.02798435,
          0.02484895, -0.04323708]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [23]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [24]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_true,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,y_pred=y_pred,average='weighted')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results

In [25]:
# Get Baseline Results
baseline_results=calculate_results(y_true=val_labels,y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# **Model1 : Feed Forward Neural Network ( Dense Model)**

In [26]:
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers

In [27]:
SAVE_DIR="ModelLogs"

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # Add this layer to reduce dimensions
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_Dense")
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_1_history=model_1.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: ModelLogs/model_1_dense/20251102-060544
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5591 - loss: 0.6820 - val_accuracy: 0.5433 - val_loss: 0.6843
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5917 - loss: 0.6650 - val_accuracy: 0.5433 - val_loss: 0.7112
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6110 - loss: 0.6558 - val_accuracy: 0.6339 - val_loss: 0.6401
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6392 - loss: 0.6358 - val_accuracy: 0.6549 - val_loss: 0.6239
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6623 - loss: 0.6130 - val_accuracy: 0.6614 - val_loss: 0.6177


In [29]:
model_1.evaluate(val_sentances,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6413 - loss: 0.6309


[0.6176876425743103, 0.6614173054695129]

In [30]:
model_1_predict_probs=model_1.predict(val_sentances)
model_1_predict_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.6000757 ],
       [0.70977825],
       [0.75046206],
       [0.41691977],
       [0.49815437],
       [0.688292  ],
       [0.7480304 ],
       [0.6953975 ],
       [0.7167872 ],
       [0.35637158],
       [0.53560066],
       [0.6004317 ],
       [0.6231699 ],
       [0.4981749 ],
       [0.66998726],
       [0.56159693],
       [0.31234413],
       [0.41813892],
       [0.49654013],
       [0.57552546],
       [0.29940167],
       [0.15579675],
       [0.45141622],
       [0.5249313 ],
       [0.5521053 ],
       [0.65720016],
       [0.19079438],
       [0.59999186],
       [0.28744087],
       [0.33875707],
       [0.52330065],
       [0.53940964],
       [0.5743014 ],
       [0.5089578 ],
       [0.5838181 ],
       [0.20324811],
       [0.6990225 ],
       [0.35140187],
       [0.31911707],
       [0.7136676 ],
       [0.44116572],
       [0.6339199 ],
       [0.4447849 ],
       [0.5792527 ],
       [0.40747023],
       [0.6502924 ],
       [0.5189293 ],
       [0.776

In [31]:
model_1_predict_probs.shape

(762, 1)

In [32]:
model_1_predict_probs[0]

array([0.6000757], dtype=float32)

In [33]:
# Make the preictions comaparable with the actaul labels
model_1_predict_probs=tf.squeeze(tf.round(model_1_predict_probs))
model_1_predict_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [34]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

# ***Calculate Model Results***

In [35]:
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_predict_probs)
model_1_results

{'accuracy': 0.6614173228346457,
 'precsion': 0.6771294565834475,
 'recall': 0.6614173228346457,
 'f1_score': 0.6602874688926432}

In [36]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

# ***Lets Visualize the embedding that Model learned using Tensorflow Projector***

In [37]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(100,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [38]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,789 (151.52 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,860 (101.02 KB)

In [39]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights,print(embed_weights.shape)

(100, 128)


(array([[-1.1413812e-04, -5.3410688e-03, -5.4580220e-03, ...,
         -3.3020612e-02, -1.8550812e-03, -1.3101141e-02],
        [-1.7413761e-01, -1.5042725e-01, -1.4110477e-01, ...,
          1.9334003e-01,  1.7851917e-01,  1.2511984e-01],
        [ 1.7920546e-01,  2.4265477e-01,  1.6065866e-01, ...,
         -1.7569101e-01, -1.2878729e-01, -1.5173501e-01],
        ...,
        [-2.3873987e-02, -6.4530991e-02, -4.8219692e-02, ...,
         -1.4051676e-04,  5.6345757e-02,  2.8983647e-02],
        [ 4.4057691e-01,  4.3230417e-01,  3.0093622e-01, ...,
         -3.6259872e-01, -3.0652872e-01, -3.1726417e-01],
        [ 2.6836547e-01,  3.3662140e-01,  2.2217081e-01, ...,
         -2.5594923e-01, -3.2371387e-01, -2.3181626e-01]], dtype=float32),
 None)

Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).  This code is from the tensorflow documentation Here[https://www.tensorflow.org/text/guide/word_embeddings]

In [40]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [41]:
# After this download step use projector tensorflow to visualzize it
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""


"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

# ***MODEL2 =  LSTM [LONG SHORT TERM MEMORY]***

In [42]:
from tensorflow.keras import layers


In [43]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs)

In [44]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [45]:
model_2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,457 (388.50 KB)

 Trainable params: 99,457 (388.50 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model_2_history=model_2.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: ModelLogs/model_2_LSTM/20251102-060553
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 27s 107ms/step - accuracy: 0.5760 - loss: 0.6838 - val_accuracy: 0.5433 - val_loss: 0.6907
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 41s 107ms/step - accuracy: 0.5705 - loss: 0.6843 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 23s 106ms/step - accuracy: 0.5779 - loss: 0.6816 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 23s 107ms/step - accuracy: 0.5690 - loss: 0.6839 - val_accuracy: 0.5433 - val_loss: 0.6896
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 42s 111ms/step - accuracy: 0.5707 - loss: 0.6835 - val_accuracy: 0.5433 - val_loss: 0.6915


In [47]:
model_2_pred_probs=model_2.predict(val_sentances)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


array([[0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958],
       [0.42430958]], dtype=float32)

In [48]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [49]:
model_2_results=calculate_results(y_true=val_labels,y_pred=model_2_preds)
model_2_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

In [50]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# ***As it is clear our baseline is more accurate***

# ***Model_3 GRU (Gated Recurrent Unit)***

In [51]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs)


In [52]:
model_3.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,273 (212.00 KB)

 Trainable params: 54,273 (212.00 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [54]:
model_3_history=model_3.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_GRU")])

Saving TensorBoard log files to: ModelLogs/model_GRU/20251102-060831
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 73ms/step - accuracy: 0.5770 - loss: 0.6844 - val_accuracy: 0.5433 - val_loss: 0.6902
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step - accuracy: 0.5745 - loss: 0.6830 - val_accuracy: 0.5433 - val_loss: 0.6907
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 70ms/step - accuracy: 0.5749 - loss: 0.6822 - val_accuracy: 0.5433 - val_loss: 0.6920
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - accuracy: 0.5768 - loss: 0.6820 - val_accuracy: 0.5433 - val_loss: 0.6900
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 70ms/step - accuracy: 0.5726 - loss: 0.6829 - val_accuracy: 0.5433 - val_loss: 0.6906


In [55]:
model_3_pred_probs=model_3.predict(val_sentances)
model_3_pred_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


array([[0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.43262345],
       [0.432

In [56]:
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [57]:
model_3_results=calculate_results(val_labels,model_3_pred)
model_3_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

# ***GRU Does not beat the baseline***
lets try something else

# ***Model_4 Bidirectional Model***

In [58]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_4=tf.keras.Model(inputs,outputs)

model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_4.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,745 (436.50 KB)

 Trainable params: 111,745 (436.50 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model_4_history=model_4.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bi_directional")])

Saving TensorBoard log files to: ModelLogs/model_4_bi_directional/20251102-060952
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 114ms/step - accuracy: 0.7040 - loss: 0.5881 - val_accuracy: 0.6837 - val_loss: 0.5989
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 24s 110ms/step - accuracy: 0.7031 - loss: 0.5773 - val_accuracy: 0.6864 - val_loss: 0.5970
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 23s 107ms/step - accuracy: 0.7272 - loss: 0.5475 - val_accuracy: 0.6982 - val_loss: 0.5833
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 24s 111ms/step - accuracy: 0.7251 - loss: 0.5518 - val_accuracy: 0.7034 - val_loss: 0.5788
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 24s 111ms/step - accuracy: 0.7236 - loss: 0.5468 - val_accuracy: 0.7047 - val_loss: 0.5804


In [60]:
model_4_pred_probs=model_4.predict(val_sentances)
model_4_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


array([[0.4844711 ],
       [0.7191519 ],
       [0.63544166],
       [0.20539166],
       [0.26091385],
       [0.7225902 ],
       [0.8124794 ],
       [0.72972584],
       [0.8042381 ],
       [0.12594695]], dtype=float32)

In [61]:
model_4_pred=tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_4_results=calculate_results(val_labels,model_4_pred)

In [63]:
model_4_results

{'accuracy': 0.7047244094488189,
 'precsion': 0.7212040810430698,
 'recall': 0.7047244094488189,
 'f1_score': 0.6917692751886626}

# ***Still Not enough to beat the baseline***

**1D Convolutional Neural Network**

In [64]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,strides=1,activation='relu',padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name='Model_5_Conv1D')

In [65]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [66]:
model_5_history=model_5.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.7075 - loss: 0.5791 - val_accuracy: 0.7008 - val_loss: 0.6246
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.7139 - loss: 0.5694 - val_accuracy: 0.6982 - val_loss: 0.5932
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.7196 - loss: 0.5501 - val_accuracy: 0.7165 - val_loss: 0.5822
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.7338 - loss: 0.5387 - val_accuracy: 0.7139 - val_loss: 0.5835
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7394 - loss: 0.5359 - val_accuracy: 0.7205 - val_loss: 0.5952


In [67]:
model_5_pred_probs=model_5.predict(val_sentances)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step


array([[0.477561  ],
       [0.62688875],
       [0.52648807],
       [0.2811272 ],
       [0.3329888 ],
       [0.7451238 ],
       [0.92749715],
       [0.78566587],
       [0.7641937 ],
       [0.0489632 ]], dtype=float32)

In [68]:
model_5_pred=tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [69]:
model_5_results=calculate_results(val_labels,model_5_pred)
model_5_results

{'accuracy': 0.7204724409448819,
 'precsion': 0.7395397280540561,
 'recall': 0.7204724409448819,
 'f1_score': 0.7082082471786006}

# **Still does not beat the Baseline**
 Lets try a pretrained  model from tensorflow hub
 Using *USE: Unviversal Sentence Encoder*

In [70]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [71]:
model_url='https://tfhub.dev/google/universal-sentence-encoder/4'

In [72]:
embed_samples=embed([sample_sentance])
print(embed_samples[0][:50])

tf.Tensor(
[-0.06598653  0.04152805 -0.0442245  -0.06173505  0.02490359 -0.04019508
 -0.03293895 -0.0479527  -0.00191026  0.0318088  -0.02625422 -0.03801547
  0.05038926  0.00406027 -0.07087556  0.08218092  0.04951795  0.02018038
  0.08004149 -0.01590493 -0.05630051 -0.0137695  -0.05894558 -0.05763777
  0.05377276  0.07138202  0.02498692  0.03309285 -0.04375488  0.02464543
  0.04376036  0.04523377 -0.07225093 -0.01322527 -0.0354355  -0.07865203
 -0.00896132  0.0299764  -0.02636237 -0.01949289 -0.03340762 -0.07911043
  0.04052498 -0.03795994  0.08157994 -0.01251724 -0.01995691  0.02034376
  0.03457163 -0.00202512], shape=(50,), dtype=float32)


In [73]:
sentence_encoder_layer=hub.KerasLayer(model_url,input_shape=[],dtype=tf.string,trainable=False,
                                      name='Universal_sentence_enocoder')

In [75]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model
import numpy as np


# 1. Fix for TensorFlow-Hub + Keras 3 incompatibility

class USEEncoder(layers.Layer):
    def __init__(self, model_url, trainable=False, **kwargs):
        super().__init__(**kwargs)
        self.use_layer = hub.load(model_url)
        self.trainable = trainable

    def call(self, inputs):
        return self.use_layer(inputs)





In [76]:
# 2. Load the Universal Sentence Encoder

print("Loading Universal Sentence Encoder (this may take a few seconds)...")
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_layer = USEEncoder(model_url)




Loading Universal Sentence Encoder (this may take a few seconds)...
